In [1]:
# Cài đặt thư viện
%pip install transformers datasets evaluate sacrebleu jiwer rouge_score accelerate -q
print("Cài đặt thành công!")

Note: you may need to restart the kernel to use updated packages.
Cài đặt thành công!


In [2]:
from datasets import load_dataset

DATASET_ID = "pqthinh232/vietnamese-correction-60k"

dataset = load_dataset(DATASET_ID)

print("Data structure:")
print(dataset)

# Danh sách câu lỗi
inputs =dataset['test']
references = list(dataset['test']['target'])

README.md:   0%|          | 0.00/570 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/9.78M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/992k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/985k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Data structure:
DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['input', 'target'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['input', 'target'],
        num_rows: 5000
    })
})


In [3]:
import torch
from transformers import pipeline

# Thay link model của bạn tại đây
FINE_TUNED_ID = "pqthinh232/HCMUS-vietnamese-correction-project"
BASE_MODEL_ID = "vinai/bartpho-syllable"
DEVICE = 0 if torch.cuda.is_available() else -1

print("Đang tải các Model... (có thể mất vài phút)")
ft_pipe = pipeline("text2text-generation", model=FINE_TUNED_ID, device=DEVICE)
base_pipe = pipeline("text2text-generation", model=BASE_MODEL_ID, device=DEVICE)
print("Tải Model thành công!")

Đang tải các Model... (có thể mất vài phút)


config.json:   0%|          | 0.00/910 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

dict.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Device set to use cuda:0


Tải Model thành công!


In [5]:
from tqdm.auto import tqdm

print("🚀 Đang bắt đầu quá trình dự đoán...")

# Chuẩn bị dữ liệu đầu vào
text_inputs = list(inputs['input'])
total_samples = len(text_inputs)

# 1. Chạy dự đoán cho Fine-tuned Model với thanh tiến trình
print("\n[1/2] Đang chạy Fine-tuned Model:")
ft_preds = []
# Bọc tqdm xung quanh pipeline. Lưu ý: phải truyền total vì pipeline trả về generator
for res in tqdm(ft_pipe(text_inputs, max_length=128, num_beams=5, batch_size=64),
                total=total_samples,
                desc="FT Inference"):
    ft_preds.append(res['generated_text'])



Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


🚀 Đang bắt đầu quá trình dự đoán...

[1/2] Đang chạy Fine-tuned Model:


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


FT Inference:   0%|          | 0/5000 [00:00<?, ?it/s]

In [6]:
# 2. Chạy dự đoán cho Base Model với thanh tiến trình
print("\n[2/2] Đang chạy Base Model:")
base_preds = []
for res in tqdm(base_pipe(text_inputs, max_length=128, num_beams=5, batch_size=64),
                total=total_samples,
                desc="Base Inference"):
    base_preds.append(res['generated_text'])

print("\n✅ Đã hoàn thành dự đoán từ cả 2 model.")

Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



[2/2] Đang chạy Base Model:


Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Base Inference:   0%|          | 0/5000 [00:00<?, ?it/s]


✅ Đã hoàn thành dự đoán từ cả 2 model.


In [9]:
import json
import pandas as pd
from jiwer import cer, wer
import sacrebleu
import evaluate
from tqdm import tqdm
from rouge_score import rouge_scorer

# 1. Khởi tạo các công cụ
# Dùng để tính tổng quát (Corpus level) - Chính xác hơn
cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("sacrebleu")

# Dùng để tính từng dòng (Sentence level) - Nhanh hơn
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

def get_corpus_metrics(preds, refs, name="Model"):
    """Tính toán chỉ số trung bình trên toàn bộ 60k câu"""
    print(f"--- Đang tính toán tổng quát cho {name} ---")
    b = bleu_metric.compute(predictions=preds, references=[[r] for r in refs])['score']
    c = cer_metric.compute(predictions=preds, references=refs)
    w = wer_metric.compute(predictions=preds, references=refs)
    r = rouge_metric.compute(predictions=preds, references=refs)['rouge1']
    return {
        "BLEU (↑)": round(b, 2),
        "CER (↓)": round(c, 4),
        "WER (↓)": round(w, 4),
        "F1-Rouge1 (↑)": round(r, 4)
    }

def calculate_row_metrics_fast(predict, reference):
    """Tính metrics cho đúng 1 dòng"""
    try:
        b_row = sacrebleu.sentence_bleu(predict, [reference]).score
        c_row = cer(reference, predict)
        w_row = wer(reference, predict)
        f1_row = scorer.score(reference, predict)['rouge1'].fmeasure
        return {"bleu": round(b_row, 2), "cer": round(c_row, 4), "wer": round(w_row, 4), "f1": round(f1_row, 4)}
    except:
        return {"bleu": 0, "cer": 1.0, "wer": 1.0, "f1": 0}

# --- BƯỚC 1: TÍNH TỔNG QUÁT (CORPUS LEVEL) ---
print("1. Đang tính chỉ số trung bình toàn tập dữ liệu...")
stats_baseline = get_corpus_metrics(base_preds, references, "Baseline")
stats_finetuned = get_corpus_metrics(ft_preds, references, "Fine-tuned")

# Hiển thị bảng so sánh ngay lập tức
df_summary = pd.DataFrame([stats_baseline, stats_finetuned], index=["Baseline", "Fine-tuned"])
print("\n=== KẾT QUẢ TỔNG QUÁT TRÊN 5K MẪU ===")
display(df_summary) # Lệnh này hiện bảng đẹp trong Notebook

# --- BƯỚC 2: TÍNH CHI TIẾT TỪNG DÒNG VÀ LƯU JSON ---
print(f"\n2. Đang xử lý chi tiết từng câu để xuất JSON...")
baseline_json = []
finetuned_json = []

for i in tqdm(range(len(references)), desc="Exporting"):
    ref = references[i]
    inp = inputs['input'][i]
    
    # Baseline
    m_base = calculate_row_metrics_fast(base_preds[i], ref)
    baseline_json.append({"id": i+1, "input": inp, "target": ref, "predict": base_preds[i], "metrics": m_base})
    
    # Fine-tuned
    m_ft = calculate_row_metrics_fast(ft_preds[i], ref)
    finetuned_json.append({"id": i+1, "input": inp, "target": ref, "predict": ft_preds[i], "metrics": m_ft})

# Ghi file
with open('result_baseline_full.json', 'w', encoding='utf-8') as f:
    json.dump(baseline_json, f, ensure_ascii=False, indent=2)
with open('result_finetuned_full.json', 'w', encoding='utf-8') as f:
    json.dump(finetuned_json, f, ensure_ascii=False, indent=2)

print("\n Đã có cả bảng tổng kết và file chi tiết.")

1. Đang tính chỉ số trung bình toàn tập dữ liệu...
--- Đang tính toán tổng quát cho Baseline ---
--- Đang tính toán tổng quát cho Fine-tuned ---

=== KẾT QUẢ TỔNG QUÁT TRÊN 5K MẪU ===


,BLEU (↑),CER (↓),WER (↓),F1-Rouge1 (↑)
Baseline,32.97,0.5859,0.8174,0.7421
Fine-tuned,89.05,0.0201,0.0564,0.9825



2. Đang xử lý chi tiết từng câu để xuất JSON...


Exporting: 100%|██████████| 5000/5000 [00:08<00:00, 578.40it/s]



 Đã có cả bảng tổng kết và file chi tiết.
